# Demo workflows

Below are demonstrated the use of this library as a Python package and from the command line interface. Note that one step demonstrated below is to save datasets you need to run the local examples.

In [1]:
import xarray as xr
import ocean_data_gateway as odg
import cf_xarray
import pandas as pd
import extract_model as em
import ocean_model_skill_assessor as omsa
import numpy as np
from IPython import display
from glob import glob

In [2]:
criteria = {
    "salt": {
        "name": "sea_water_practical_salinity$",
    },
}

var_def = {
    "salt": {"units": "psu", "fail_span": [-10, 60], "suspect_span": [-1, 45]},
}


In [3]:
url = 'https://gist.githubusercontent.com/kthyng/c3cc27de6b4449e1776ce79215d5e732/raw/af448937e4896535e36ef6522df8460e8f928cd6/my_custom_criteria.py'
criteria = odg.return_response(url)
criteria

{'ssh': {'standard_name': 'sea_surface_height$|sea_surface_elevation|sea_surface_height_above_sea_level$',
  'name': '(?i)sea_surface_elevation(?!.*?_qc)|(?i)sea_surface_height_above_sea_level_geoid_mllw$|(?i)zeta$|(?i)Sea Surface Height(?!.*?_qc)|(?i)Water Surface above Datum(?!.*?_qc)'},
 'temp': {'name': '(?i)temp$|(?i)temperature$|(?i)tem$|(?i)s.sea_water_temperature$|(?i)temperature(?!.*(skin|ground|air|_qc))'},
 'salt': {'standard_name': 'sea_water_salinity$|sea_water_practical_salinity$',
  'name': '(?i)salinity(?!.*(soil|_qc))|(?i)sea_water_salinity$|(?i)sea_water_practical_salinity$|(?i)salinity$|(?i)salt$|(?i)sal$|(?i)s.sea_water_practical_salinity$'},
 'u': {'standard_name': 'eastward_sea_water_velocity$|sea_water_x_velocity|surface_eastward_sea_water_velocity',
  'name': '(?i)eastward_sea_water_velocity(?!.*?_qc)|(?i)sea_water_x_velocity(?!.*?_qc)|(?i)uo(?!.*?_qc)'},
 'v': {'standard_name': 'northward_sea_water_velocity$|sea_water_y_velocity|surface_northward_sea_water_velo

In [ ]:
# # for ciofs:
# kw = {
#     "min_lon": -156,
#     "max_lon": -148,
#     "min_lat": 56,
#     "max_lat": 62,
#     "min_time": '2022-5-20',
#     "max_time": '2022-5-23',
# }

In [ ]:
# # setup Data search object
# data = odg.Gateway(kw=kw, approach='region')

In [ ]:
#odg.Gateway?

In [ ]:
#len(data.dataset_ids)

In [ ]:
# valid_ids = []
# valid_dss = []
# for k in range(238, len(data.dataset_ids)): 
#    if data[data.dataset_ids[k]]: 
#        if ('sea_water_salinity' in data[data.dataset_ids[k]].variables) | ('sea_water_practical_salinity' in data[data.dataset_ids[k]].variables):
#            print(k, '  ', data.dataset_ids[k])
#            valid_ids.append(k)
#            valid_dss.append(data.dataset_ids[k])           

In [ ]:
# # for ciofs
# valid_ids = [51, 83, 219, 238]
# valid_dss = ['nerrs_kacsdwq', 'cdmo_nerrs_3b00077a', 'cdmo_nerrs_3b040240', 'nerrs_kachdwq']

In [ ]:
# # plotting just stations with salinity 
# import matplotlib.pyplot as plt

# plot_lat = []
# plot_lon = []
# plot_text = []
# for valid_ds in valid_dss:
#     plot_lat.append(data[valid_ds].latitude)
#     plot_lon.append(data[valid_ds].longitude)
#     # plot_text.append(str(k))

In [ ]:
# # plotting all stations
# plt.plot(plot_lon,plot_lat,'.')
# plt.show()

# Let's test omsa.run - that's where the problem was, originally (with the netcdf file downloaded)
The problem was originally `KeyError: lat, on ocean_model_skill_assessor/main.py:370`. That's within the omsa.run func. So let's run that func piecewise here.


### Testing read_model

In [4]:
def read_model(loc_model, xarray_kwargs, time_range=None):
    """Read in model output input by user.

    Parameters
    ----------
    loc_model : str
        Relative or absolute, local or nonlocal path to model output.
    xarray_kwargs : dict, optional
        Keyword arguments to pass into `xr.open_dataset`.
    time_range: list
        [min_time, max_time] for desired time range of search where each
        are strings that can be interpreted with pandas `Timestamp`.

    Returns
    -------
    xarray Dataset containing model output.
    """

    dsm = xr.open_dataset(loc_model, **xarray_kwargs)

    # add more cf-xarray info
    dsm = dsm.cf.guess_coord_axis()

    # drop duplicate time indices if present
    # also limit the time range of the model output to what we are requesting from the data to
    # not waste extra time on the model interpolation
    # https://stackoverflow.com/questions/51058379/drop-duplicate-times-in-xarray
    _, index = np.unique(dsm.cf["T"], return_index=True)

    if time_range:
        dsm = dsm.cf.isel(T=index).cf.sel(T=slice(time_range[0], time_range[1]))

    # force longitude to be from -180 to 180
    lkey = dsm.cf["longitude"].name
    dsm[lkey] = dsm.cf["longitude"].where(
        dsm.cf["longitude"] < 180, dsm.cf["longitude"] - 360
    )

    return dsm

In [5]:
# this was done with the file downloaded from: 
# https://thredds/ncss/NOAA_COOPS_OFS_CIOFS.nc?var=salt&north=61.5247&west=-156.4852&east=-148.9251&south=56.7004&horizStride=1&time_start=2022-05-20T00%3A00%3A00Z&time_end=2022-05-23T18%3A00%3A00Z&timeStride=1&vertCoord=1&vertStride=1&addLatLon=true&accept=netcdf
model_url = '/home/anapaula/Downloads/NOAA_COOPS_OFS_CIOFS.nc'
loc_model = model_url
# xarray_kwargs = xarray_kwargs={'chunks': {'time': 1, 'depth': 1},"decode_coords":"all"}
xarray_kwargs = xarray_kwargs={'chunks': {'time': 1, 'depth': 1}}
time_range=['2022-5-20','2022-5-23']

In [6]:
dsm = read_model(loc_model, xarray_kwargs, time_range)
# dsm = read_model(loc_model, time_range)

In [7]:
dsm
# dsm.dims
# dsm.coords
# dsm.cf
# dsm.lon_rho.values

<xarray.Dataset>
Dimensions:     (ocean_time: 67, s_rho: 1, eta_rho: 1044, xi_rho: 724)
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2022-05-21 ... 2022-05-23T18:00:00
  * s_rho       (s_rho) float32 -0.01667
    lat_rho     (eta_rho, xi_rho) float32 dask.array<chunksize=(1044, 724), meta=np.ndarray>
    lon_rho     (eta_rho, xi_rho) float32 dask.array<chunksize=(1044, 724), meta=np.ndarray>
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    salt        (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(67, 1, 1044, 724), meta=np.ndarray>
    Cs_r        (s_rho) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    zeta        (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(67, 1044, 724), meta=np.ndarray>
    h           (eta_rho, xi_rho) float32 dask.array<chunksize=(1044, 724), meta=np.ndarray>
    hc          float32 ...
Attributes: (12/67)
    file:                          nos.ciofs.fields.forecast.20220621.t18z_00...
    format:                        netCDF-4/HDF5 file
    type:                          ROMS/TOMS history file
    var_info:                      varinfo.dat
    rst_file:                      nos.ciofs.rst.forecast.20220621.t18z.nc
    his_base:                      nos.ciofs.fields.forecast.20220621.t18z
    ...                            ...
    summary:                       The Cook Inlet Ocean Forecast System uses ...
    time_coverage_duration:        PT24H
    time_coverage_end:             2022-06-23T23:00:00
    time_coverage_resolution:      PT1H
    time_coverage_start:           2022-06-23T00:00:00
    History:                       Translated to CF-1.0 Conventions by Netcdf...

### read_model works, good. Now let's test ocean data gateway for stations - that's where the problem was, originally (with the netcdf file downloaded)



In [8]:
# test ocean data gateway

# best datasets - ciofs
dataset_ids = ['nerrs_kacsdwq', 'cdmo_nerrs_3b00077a']

parallel = False
variables=['salt']
readers=[odg.erddap]
local=None
erddap={'known_server': 'ioos'}
axds=None
skip_units=False

kwargs = dict(
        criteria=criteria,
        var_def=var_def,
        approach='stations',
        parallel=parallel,
        variables=variables,
        readers=readers,
        local=local,
        erddap=erddap,
        axds=axds,
        skip_units=skip_units,
    )

time_range=['2022-5-20','2022-5-23']
kw = dict(min_time=time_range[0], max_time=time_range[1])
kwargs["kw"] = kw

kwargs["stations"] = dataset_ids

search = odg.Gateway(**kwargs)

In [ ]:
# check what's available
# search.kwargs
# search.kwargs_all
# search.criteria
# search.var_def
# search.sources
# search.dataset_ids

### Ocean data gateway works, good. Now let's make sure prep extract model works

In [9]:
def prep_em(input_data):
    """Prepare to run extract_model."""

    if isinstance(input_data, pd.DataFrame):
        data = input_data
        tname = data.cf["T"].name
        data[tname] = pd.to_datetime(data.cf["T"])
        data = data.set_index(data.cf["T"])
    else:
        data = input_data
    lon = float(data.cf["longitude"].values[0])
    lat = float(data.cf["latitude"].values[0])
    T = None
    # only compare surface
    Z = None

    return data, lon, lat, T, Z

In [10]:
data, lon, lat, T, Z = prep_em(search[dataset_ids[0]])

In [ ]:
data

In [ ]:
lon

In [ ]:
data

# great, that works too. Now let's check the kwargs for em.select   
            kwargs = dict(
                da=dsm.cf[variable].cf.isel(Z=0).cf.sel(lon=slice(lon - 5, lon + 5), lat=slice(lat - 5, lat + 5)),
                longitude=lon,
                latitude=lat,
                T=T,
                iZ=Z,
                locstream=True,
            )   
            
The main issue here is with subsetting the model. Ciofs dataset has as dimensions ocean_time, s_rho, eta_rho and xi_rho. And the coordinates are ocean_time, s_rho, lat_rho, and lon_rho.
So we can't subset directly on lat_rho and lon_rho.

In [ ]:
dsm

In [ ]:
dsm.salt.cf.sel(longitude=slice(lon - 5, lon + 5), latitude=slice(lat - 5, lat + 5))

In [ ]:
# KeyError: 'no index found for coordinate lat_rho'

In [ ]:
dsm.salt.cf.sel(lon=slice(lon - 5, lon + 5), lat=slice(lat - 5, lat + 5))

In [ ]:
# KeyError: 'lat is not a valid dimension or coordinate'

In [ ]:
dsm.cf['latitude']

In [ ]:
dsm.cf['longitude']

In [ ]:
lon, lat

## those didn't work; let's try something else

In [16]:
variable=['salt']            
da=dsm.cf[variable].cf.isel(Z=0)
kwargs = dict(da=dsm.cf[variable].cf.isel(Z=0),
              longitude=lon,
              latitude=lat,
              T=T,
              iZ=Z,
              locstream=True,
              )
model_var = em.select(**kwargs).to_dataset()

ValueError: ESMC_FieldRegridStore failed with rc = 506. Please check the log files (named "*ESMF_LogFile").

In [17]:
da

<xarray.Dataset>
Dimensions:     (ocean_time: 67, eta_rho: 1044, xi_rho: 724)
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2022-05-21 ... 2022-05-23T18:00:00
    s_rho       float32 -0.01667
    lat_rho     (eta_rho, xi_rho) float32 dask.array<chunksize=(1044, 724), meta=np.ndarray>
    lon_rho     (eta_rho, xi_rho) float32 dask.array<chunksize=(1044, 724), meta=np.ndarray>
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    salt        (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(67, 1044, 724), meta=np.ndarray>
Attributes: (12/67)
    file:                          nos.ciofs.fields.forecast.20220621.t18z_00...
    format:                        netCDF-4/HDF5 file
    type:                          ROMS/TOMS history file
    var_info:                      varinfo.dat
    rst_file:                      nos.ciofs.rst.forecast.20220621.t18z.nc
    his_base:                      nos.ciofs.fields.forecast.20220621.t18z
    ...                            ...
    summary:                       The Cook Inlet Ocean Forecast System uses ...
    time_coverage_duration:        PT24H
    time_coverage_end:             2022-06-23T23:00:00
    time_coverage_resolution:      PT1H
    time_coverage_start:           2022-06-23T00:00:00
    History:                       Translated to CF-1.0 Conventions by Netcdf...

## Ok so for the case below, em.select does work, but it can't convert the result of em.select to a dataset

In [21]:
del da
variable=['salt']        
da = dsm.where((dsm.lon_rho > lon-2) & (dsm.lon_rho < lon+2) & (dsm.lat_rho > lat-2) & (dsm.lat_rho < lat+2), drop=True).cf.isel(Z=0)

kwargs = dict(da=da.cf[variable],
              longitude=lon,
              latitude=lat,
              T=T,
              iZ=Z,
              locstream=True,
              )
model_var = em.select(**kwargs)
auxx = model_var.to_dataset()

AttributeError: 'Dataset' object has no attribute 'to_dataset'

In [22]:
model_var

<xarray.Dataset>
Dimensions:     (ocean_time: 67, locations: 1)
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2022-05-21 ... 2022-05-23T18:00:00
    s_rho       float32 -0.01667
    lon         (locations) float64 -151.7
    lat         (locations) float64 59.44
Dimensions without coordinates: locations
Data variables:
    salt        (ocean_time, locations) float32 dask.array<chunksize=(67, 1), meta=np.ndarray>
Attributes: (12/68)
    file:                          nos.ciofs.fields.forecast.20220621.t18z_00...
    format:                        netCDF-4/HDF5 file
    type:                          ROMS/TOMS history file
    var_info:                      varinfo.dat
    rst_file:                      nos.ciofs.rst.forecast.20220621.t18z.nc
    his_base:                      nos.ciofs.fields.forecast.20220621.t18z
    ...                            ...
    time_coverage_duration:        PT24H
    time_coverage_end:             2022-06-23T23:00:00
    time_coverage_resolution:      PT1H
    time_coverage_start:           2022-06-23T00:00:00
    History:                       Translated to CF-1.0 Conventions by Netcdf...
    regrid_method:                 bilinear

In [23]:
lon, lat

(-151.72096, 59.44099)

In [19]:
da

<xarray.Dataset>
Dimensions:     (ocean_time: 67, eta_rho: 262, xi_rho: 551)
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2022-05-21 ... 2022-05-23T18:00:00
    s_rho       float32 -0.01667
    lat_rho     (eta_rho, xi_rho) float32 dask.array<chunksize=(262, 551), meta=np.ndarray>
    lon_rho     (eta_rho, xi_rho) float32 dask.array<chunksize=(262, 551), meta=np.ndarray>
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    salt        (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(67, 262, 551), meta=np.ndarray>
    Cs_r        (eta_rho, xi_rho) float32 dask.array<chunksize=(262, 551), meta=np.ndarray>
    zeta        (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(67, 262, 551), meta=np.ndarray>
    h           (eta_rho, xi_rho) float32 dask.array<chunksize=(262, 551), meta=np.ndarray>
    hc          (eta_rho, xi_rho) float64 dask.array<chunksize=(262, 551), meta=np.ndarray>
Attributes: (12/67)
    file:                          nos.ciofs.fields.forecast.20220621.t18z_00...
    format:                        netCDF-4/HDF5 file
    type:                          ROMS/TOMS history file
    var_info:                      varinfo.dat
    rst_file:                      nos.ciofs.rst.forecast.20220621.t18z.nc
    his_base:                      nos.ciofs.fields.forecast.20220621.t18z
    ...                            ...
    summary:                       The Cook Inlet Ocean Forecast System uses ...
    time_coverage_duration:        PT24H
    time_coverage_end:             2022-06-23T23:00:00
    time_coverage_resolution:      PT1H
    time_coverage_start:           2022-06-23T00:00:00
    History:                       Translated to CF-1.0 Conventions by Netcdf...

In [ ]:
whos

Ok so that doesn't work either; something to do with regridding. 

In [ ]:
dsm = dsm.where((dsm.lon_rho > lon-1) & (dsm.lon_rho < lon+1) & (dsm.lat_rho > lat-1) & (dsm.lat_rho < lat+1), drop=True)

In [ ]:
dsm

In [ ]:

model_var = em.select(**kwargs).to_dataset()

In [ ]:
da = dsm.where((dsm.lon_rho > lon-1) & (dsm.lon_rho < lon+1) & (dsm.lat_rho > lat-1) & (dsm.lat_rho < lat+1), drop=True).cf.isel(Z=0)

In [ ]:
da

In [ ]:
whos

In [ ]:
# To subset, we need to find the indices of eta_rho and xi_rho with values for lat and lon that interest us. The fact that this is not a rectilinear grid makes it more complicated.

In [ ]:
dsm.salt.isel(xi_rho=0,eta_rho=100,ocean_time=10) # selecting with indices

In [ ]:
dsm.salt.sel(xi_rho=0,eta_rho=100,ocean_time='2022-05-23T17:00:00.000000000') # selecting by values

In [ ]:
# Ok, now let's try to find something that works for us

In [ ]:
# find indices with nice lats and lons
lati = np.array(np.where((dsm.lat_rho > lat-5) & (dsm.lat_rho < lat+5)),dsm.lat_rho)
loni = np.array(np.where((dsm.lon_rho > lon-5) & (dsm.lon_rho < lon+5)),dsm.lon_rho)


l, c = np.array(np.where((dsm.cf['latitude'] > lat-5) & (dsm.cf['latitude'] < lat+5) & (dsm.cf['longitude'] > lon-5) & (dsm.cf['longitude'] < lon+5)))
# l and c each returns an array with the number of elements in the original matrix

# so we can't do something like
aux=dsm.salt[:,:,l,c] # dimensions (ocean_time, s_rho, eta_rho, xi_rho)

Now let's try the func from https://github.com/xoceanmodel/xroms

In [ ]:
def subset(ds, X=None, Y=None):
    """Subset model output horizontally using isel, properly accounting for horizontal grids.
    Inputs
    ------
    ds: xarray Dataset
        Dataset of ROMS model output. Assumes that full regular grid setup is
        available and has been read in using xroms so that dimension names
        have been updated.
    X: slice, optional
        Slice in X dimension using form `X=slice(start, stop, step)`. For example,
        >>> X=slice(20,40,2)
        Indices are used for rho grid, and psi grid is reduced accordingly.
    Y: slice, optional
        Slice in Y dimension using form `Y=slice(start, stop, step)`. For example,
        >>> Y=slice(20,40,2)
        Indices are used for rho grid, and psi grid is reduced accordingly.
    Returns
    -------
    Dataset with form as if model had been run at the subsetted size. That is, the outermost
    cells of the rho grid are like ghost cells and the psi grid is one inward from this size
    in each direction.
    Notes
    -----
    X and Y must be slices, not single numbers.
    Example usage
    -------------
    Subset only in Y direction:
    >>> xroms.subset(ds, Y=slice(50,100))
    Subset in X and Y:
    >>> xroms.subset(ds, X=slice(20,40), Y=slice(50,100))
    """

#     if X is not None:
#         assert isinstance(X, slice), "X must be a slice, e.g., slice(50,100)"
#         ds = ds.isel(xi_rho=X, xi_u=slice(X.start, X.stop - 1))

#     if Y is not None:
#         assert isinstance(Y, slice), "Y must be a slice, e.g., slice(50,100)"
#         ds = ds.isel(eta_rho=Y, eta_v=slice(Y.start, Y.stop - 1))

    # I believe I need to edit this
    if X is not None:
        assert isinstance(X, slice), "X must be a slice, e.g., slice(50,100)"
        ds = ds.isel(eta_rho=slice(Y.start, Y.stop - 1))

    if Y is not None:
        assert isinstance(Y, slice), "Y must be a slice, e.g., slice(50,100)"
        ds = ds.isel(xi_rho=slice(X.start, X.stop - 1))
        
        
    return ds

In [ ]:
#find index of xi_rho of min lon
import numpy as np

# abslat = np.abs(dsm.lat_rho-lat)


# lonmaxabs = np.abs(dsm.lon_rho-(lon+5))


# dsm.cf['longitude']
# minlon = np.abs(dsm.cf['longitude'][0,:]-(lon-5))
# np.where(aa == np.min(aa).values)



# dsm.loc[dict(xi_rho=slice(lon-5,lat+5))]

# X = slice
# dsm.salt.isel(xi_rho = X)

In [ ]:
ks = aa[0][:]
ls = aa[1][:]
for k, l in [ks, ls]:
    print('oba')
    #print(dsm.lon_rho[k,l])

In [ ]:
# lonminabs = np.abs(dsm.lon_rho-(lon-5))
# aa = np.where(lonminabs == lonminabs.values.min())
# # get the second dimension
# xi_rho_min = aa[1][0] # we want column number here

lonmaxabs = np.abs(dsm.lon_rho-(lon+5))
aa = np.where(lonmaxabs == lonmaxabs.values.max())
# get the second dimension
xi_rho_max = aa[1][0] # we want column number here

for item in zip(aa[0][:],aa[1][:]):
    print(item)
    # print(dsm.lon_rho[item[0],item[1]].values)

In [ ]:
model_var = em.select(**kwargs).to_dataset()

In [ ]:
xi_rho_max

In [ ]:
item[1]

In [ ]:
ks

In [ ]:
np.min(lonminabs).values

In [ ]:
lonminabs

In [ ]:
subset(dsm, X=slice(lon - 5, lon + 5), Y=slice(lat - 5, lat + 5))

# some more tests/junk

In [ ]:
# First, find the index of the grid point nearest a specific lat/lon.   
abslat = np.abs(dsm.lat_rho-lat)
abslon = np.abs(dsm.lon_rho-lon)

import numpy as np
dsm.cf['longitude']
minlon = np.abs(dsm.cf['longitude'][0,:]-(lon-5))
np.where(aa == np.min(aa).values)

maxlon = np.abs(dsm.cf['longitude'][0,:]-(lon+5))

In [ ]:
lat2 = dsm.lat_rho.values.ravel()
lon2 = dsm.lon_rho.values.ravel()

indices = np.array(np.where((lat2 > lat-5) & (lat2 < lat+5) & (lon2 > lon-5) & (lon2 < lon+5)))

for time in ocean_time:
    dsm.salt[time,0,...].ravel()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import path 
import netCDF4

def bbox2ij(lon,lat,bbox=[-160., -155., 18., 23.]):
    """Return indices for i,j that will completely cover the specified bounding box.     
    i0,i1,j0,j1 = bbox2ij(lon,lat,bbox)
    lon,lat = 2D arrays that are the target of the subset
    bbox = list containing the bounding box: [lon_min, lon_max, lat_min, lat_max]

    Example
    -------  
    >>> i0,i1,j0,j1 = bbox2ij(lon_rho,[-71, -63., 39., 46])
    >>> h_subset = nc.variables['h'][j0:j1,i0:i1]       
    """
    bbox=np.array(bbox)
    mypath=np.array([bbox[[0,1,1,0]],bbox[[2,2,3,3]]]).T
    p = path.Path(mypath)
    points = np.vstack((lon.values.ravel(),lat.values.ravel())).T   
    n,m = np.shape(lon)
    inside = p.contains_points(points).reshape((n,m))
    ii,jj = np.meshgrid(xrange(m),xrange(n))
    return min(ii[inside]),max(ii[inside]),min(jj[inside]),max(jj[inside])

In [ ]:
bbox = [lon-5, lon+5, lat-5, lat+5]
i0,i1,j0,j1 = bbox2ij(dsm.lon_rho,dsm.lat_rho,bbox)

In [ ]:
dsm.lon_rho.values.ravel()

In [ ]:
dsm.salt.isel(eta_rho=l,xi_rho=c)


# ds = subset(dsm,X=slice(lon - 5, lon + 5), Y=slice(lat - 5, lat + 5))

In [ ]:
# a = np.arange(10)
# np.where(a < 5)
a = np.array([[0, 1, 2],
              [0, 2, 4],
              [0, 3, 6]])
np.where(a < 4)

In [ ]:
dsm.lat_rho.max().values

In [ ]:
lat

In [ ]:
whos

In [ ]:
whos

In [ ]:
aux.shape

In [ ]:
whos

In [ ]:
dsm.lat_rho[aux].values


In [ ]:
aa.values

In [ ]:
np.min(aa).values

In [ ]:
np.where(aa == np.min(aa).values) # primeiro eh eta_rho, depois eh xi_rho

In [ ]:
aa.shape

In [ ]:
dsm.lat_rho[0,:].values

In [ ]:
aa.values

In [ ]:
dsm.rename_dims({"s_rho":"z","eta_rho":"lon_rho","xi_rho":"lat_rho"})
dsm.swap_dims({"s_rho":"z","eta_rho":"lon_rho","xi_rho":"lat_rho"})
# can't rename the dim b/c lon_rho and lat/rho are 2d

In [ ]:
dsm.sel(eta_rho=slice(lon - 5, lon + 5),method="nearest")

## Python library

This package can be used as a Python package or from the command line. Here we demonstrate its use as a Python package.

### Nonlocal reader: stations

This demonstrates the case in which your data files for comparing with model output are not available locally, but you know their names and where to find them (in this case, at the IOOS ERDDAP server). More information about these inputs is available in the [API docs](https://ocean-model-skill-assessor.readthedocs.io/en/latest/api.html).

In [ ]:
var_def

In [ ]:
criteria

In [ ]:
from ocean_data_gateway import read_model

In [ ]:
# FOR CIOFS:

# model_url = 'http://thredds.aoos.org/thredds/dodsC/NOAA_COOPS_OFS_CIOFS.nc'
model_url = 'NOAA_COOPS_OFS_CIOFS.nc'


# best datasets - ciofs
dataset_ids = ['nerrs_kacsdwq', 'cdmo_nerrs_3b00077a'] 
    
search = omsa.run(loc_model = model_url,
                  approach='stations',
                  criteria=criteria,
                  var_def=var_def,
                  xarray_kwargs={'chunks': {'time': 1, 'depth': 1}},
                  time_range=['2022-5-20','2022-5-23'], 
                  variables=['salt'],
                  readers=[odg.erddap],
                  erddap={
                      'known_server': 'ioos'
                  },
                  stations = dataset_ids,
                  figname_map='nonlocal_library.png',
                  figname_data_prefix='nonlocal_library_',
                  parallel=False
)

In [ ]:
# Error:Attribute value out of range: _FillValue = NaN
# Error:Attribute value out of range: _FillValue = NaN
# Error:Attribute value out of range: _FillValue = NaN

# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
# Input In [10], in <cell line: 8>()
#       5 # best datasets - ciofs
#       6 dataset_ids = ['nerrs_kacsdwq', 'cdmo_nerrs_3b00077a'] 
# ----> 8 search = omsa.run(loc_model = model_url,
#       9                   approach='stations',
#      10                   criteria=criteria,
#      11                   var_def=var_def,
#      12                   xarray_kwargs={'chunks': {'time': 1, 'depth': 1}},
#      13                   time_range=['2022-5-20','2022-5-23'], 
#      14                   variables=['salt'],
#      15                   readers=[odg.erddap],
#      16                   erddap={
#      17                       'known_server': 'ioos'
#      18                   },
#      19                   stations = dataset_ids,
#      20                   figname_map='nonlocal_library.png',
#      21                   figname_data_prefix='nonlocal_library_',
#      22                   parallel=False
#      23 )

# File ~/ocean-model-skill-assessor/ocean_model_skill_assessor/main.py:334, in run(approach, loc_model, axds, bbox, criteria, erddap, figname_map, figname_data_prefix, local, only_search, only_searchplot, parallel, readers, run_qc, skip_units, stations, time_range, variables, var_def, xarray_kwargs)
#     331     return search
#     333 # Plot discovered datasets
# --> 334 lls_stations, names_stations, lls_box, names_boxes = prep_plot(search)
#     335 omsa.map.plot(
#     336     lls_stations=lls_stations,
#     337     names_stations=names_stations,
#    (...)
#     342     figname=figname_map,
#     343 )
#     345 if only_searchplot:

# File ~/ocean-model-skill-assessor/ocean_model_skill_assessor/main.py:126, in prep_plot(search)
#     123 def prep_plot(search):
#     124     """Put together inputs for map plot."""
# --> 126     sub = search.meta.loc[
#     127         search.dataset_ids,
#     128         [
#     129             "geospatial_lon_min",
#     130             "geospatial_lat_min",
#     131             "geospatial_lon_max",
#     132             "geospatial_lat_max",
#     133         ],
#     134     ]
#     135     lls = sub.values
#     136     names = list(sub.index.values)

# File ~/miniconda3/envs/ocean-model-skill-assessor/lib/python3.10/site-packages/ocean_data_gateway/gateway.py:349, in Gateway.meta(self)
#     347 meta = []
#     348 for source in self.sources:
# --> 349     meta.append(source.meta)
#     351 # self._meta = meta
#     352 # merge metadata into one DataFrame
#     353 self._meta = pd.concat(meta, axis=0, join="outer")

# File ~/miniconda3/envs/ocean-model-skill-assessor/lib/python3.10/site-packages/ocean_data_gateway/readers/erddap.py:402, in ErddapReader.meta(self)
#     400     downloads = []
#     401     for dataset_id in self.dataset_ids:
# --> 402         downloads.append(self.meta_by_dataset(dataset_id))
#     404 # make dict from individual dicts
#     405 from collections import ChainMap

# File ~/miniconda3/envs/ocean-model-skill-assessor/lib/python3.10/site-packages/ocean_data_gateway/readers/erddap.py:366, in ErddapReader.meta_by_dataset(self, dataset_id)
#     362     download_url = self.e.get_download_url(response="opendap")
#     364 # check if "prediction" is present in metadata, esp in case of NOAA
#     365 # model predictions
# --> 366 is_prediction = "Prediction" in " ".join(
#     367     list(info["Value"].replace(np.nan, None).values)
#     368 )
#     370 # add erddap server name
#     371 return {
#     372     dataset_id: [self.e.server, download_url, info_url, is_prediction]
#     373     + items
#     374     + [self.variables]
#     375 }

# TypeError: sequence item 56: expected str instance, NoneType found

In [ ]:
# error KeyError: 'lat is not a valid dimension or coordinate'
# from the ciofs files:
#     <class 'netCDF4._netCDF4.Variable'>
# float32 lat_rho(eta_rho, xi_rho)
#     field: lat_rho, scalar
#     long_name: latitude of RHO-points
#     standard_name: latitude
#     units: degree_north
#     _FillValue: nan
#     missing_value: nan
# unlimited dimensions: 
# current shape = (1044, 724)
# filling on

#### Save local data files

Here we save the data files to use for the local examples.

In [ ]:
# CIOFS dataset_ids = ['nerrs_kacsdwq', 'cdmo_nerrs_3b00077a'] 
search['nerrs_kacsdwq'].squeeze().to_dataframe().to_csv('nerrs_kacsdwq')
search['cdmo_nerrs_3b00077a'].to_netcdf('cdmo_nerrs_3b00077a.nc')


### Nonlocal reader: region

This demonstrates the case in which your data files for comparing with model output are not available locally, and you want to perform a search in time and space. By default this would search in the spatial bounding box of the model output, but here we instead input a smaller bounding box so as to limit the number of datasets found and used. For several of the datasets, the model output isn't available (must be determined to be on land). 

In [ ]:
omsa.set_criteria(criteria)

bbox = [-153, 57, -145, 65]
search = omsa.run(
                  loc_model=model_url,
                  approach='region',
                  bbox=bbox,
                  criteria=criteria,
                  var_def=var_def,
                  xarray_kwargs={'chunks': {'time': 1, 'depth': 1}},
                  time_range=['2022-5-20','2022-5-23'], 
                  variables=['salt'],
                  readers=[odg.erddap],
                  erddap={
                      'known_server': 'ioos'
                  },
                  figname_map='nonlocal_library_region.png',
                  figname_data_prefix='nonlocal_library_region_'
)

### Local reader

This demonstrates the case in which your data files for comparing with model output are available locally.

In [ ]:
omsa.set_criteria(criteria)

filenames = [
             'nerrs_kacsdwq.csv',
             'noaa_nos_co_ops_9454050.csv',
            ]

skip_units = True

search = omsa.run(
                  loc_model='hycomglb93.0_model_output.nc',
                  approach='region',
                  criteria=criteria,
                  var_def=var_def,
                  skip_units=skip_units,
                  xarray_kwargs={'chunks': {'time': 1, 'depth': 1}},
                  time_range=['2022-4-15','2022-5-4'],  
                  variables=['salt'],
                  readers=[odg.local],
                  local={'filenames': filenames},
                  figname_map='local_library.png',
                  figname_data_prefix='local_library_'
)

## Command line interface

Here we demonstrate the use of the command line interface mode of the package. The config yaml file must be modified for the necessary inputs. These are the same examples as above, but accomplished via the command line interface instead of the Python package.

### Local reader

In [ ]:
config_file = 'config_local.yaml'

In [ ]:
# %load config_local.yaml
---
approach: "stations"
loc_model: "https://thredds.cencoos.org/thredds/dodsC/CENCOOS_CA_ROMS_FCST.nc"
axds:
bbox:
criteria:
  salt:
    name:
      "sea_water_practical_salinity$"
erddap:
figname_map: "local_cli.png"
figname_data_prefix: "local_cli_"
local:
  filenames:
    - "edu_humboldt_tdp.csv"
    - "bodega-bay-bml_wts.nc"
only_search: false
only_searchplot: false
parallel: true
readers:
  - "local"
run_qc: false
skip_units: true
stations:
time_range:
  - "2021-9-1"
  - "2021-9-8"
var_def:
  salt:
    units: "psu"
    fail_span:
      - -10
      - 60
    suspect_span:
      - -1
      - 45
variables:
  - "salt"
xarray_kwargs:
  chunks:
    time:
      1
    depth:
      1


In [ ]:
!python ../ocean_model_skill_assessor/CLI.py $config_file

In [ ]:
display.Image("local_cli.png")

In [ ]:
display.Image('local_cli_edu_humboldt_tdp.csv_salt.png')

In [ ]:
display.Image('local_cli_bodega-bay-bml_wts.nc_salt.png')

### Nonlocal workflow: stations

In [ ]:
config_file = 'config_nonlocal.yaml'

In [ ]:
# %load config_nonlocal.yaml
---
approach: "stations"
loc_model: "https://thredds.cencoos.org/thredds/dodsC/CENCOOS_CA_ROMS_FCST.nc"
axds:
bbox:
criteria:
  salt:
    name:
      "sea_water_practical_salinity$"
erddap:
  known_server:
    "ioos"
figname_map: "nonlocal_cli.png"
figname_data_prefix: "nonlocal_cli_"
local:
only_search: false
only_searchplot: false
parallel: true
readers:
  - "erddap"
run_qc: false
skip_units: true
stations:
  - "edu_humboldt_tdp"
  - "bodega-bay-bml_wts"
time_range:
  - "2021-9-1"
  - "2021-9-8"
var_def:
  salt:
    units: "psu"
    fail_span:
      - -10
      - 60
    suspect_span:
      - -1
      - 45
variables:
  - "salt"
xarray_kwargs:
  chunks:
    time:
      1
    depth:
      1


In [ ]:
whos

In [ ]:
!python ../ocean_model_skill_assessor/CLI.py $config_file

In [ ]:
display.Image("nonlocal_cli.png")

In [ ]:
display.Image('nonlocal_cli_edu_humboldt_tdp_salt.png')

In [ ]:
display.Image('nonlocal_cli_bodega-bay-bml_wts_salt.png')

### Nonlocal reader: region

In [ ]:
config_file = 'config_nonlocal_region.yaml'

In [ ]:
# %load config_nonlocal_region.yaml
---
approach: "region"
loc_model: "https://thredds.cencoos.org/thredds/dodsC/CENCOOS_CA_ROMS_FCST.nc"
axds:
bbox:
  - -124.5
  - 40
  - -123.5
  - 42
criteria:
  salt:
    name:
      "sea_water_practical_salinity$"
erddap:
  known_server:
    "ioos"
figname_map: "nonlocal_cli_region.png"
figname_data_prefix: "nonlocal_cli__region"
local:
only_search: false
only_searchplot: false
parallel: true
readers:
  - "erddap"
run_qc: false
skip_units: true
stations:
time_range:
  - "2021-9-1"
  - "2021-9-8"
var_def:
  salt:
    units: "psu"
    fail_span:
      - -10
      - 60
    suspect_span:
      - -1
      - 45
variables:
  - "salt"
xarray_kwargs:
  chunks:
    time:
      1
    depth:
      1

In [ ]:
!python ../ocean_model_skill_assessor/CLI.py $config_file

In [ ]:
display.Image("nonlocal_cli_region.png")

In [ ]:
display.Image('nonlocal_cli__regionexploratorium-pco2-buoy_salt.png')

In [ ]:
display.Image('nonlocal_cli__regionedu_calpoly_marine_morro_salt.png')

In [ ]:
display.Image('nonlocal_cli__regionsccoos_ucsd_salt.png')